This notebook allow you to list events of an day and then add transportation time for each of them

[Python quickstart for calendar APi](https://developers.google.com/workspace/calendar/api/quickstart/python)

In [ ]:
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build

# Define the scope
SCOPES = ['https://www.googleapis.com/auth/calendar']

# Create the flow using the client secrets file
flow = InstalledAppFlow.from_client_secrets_file(
    'credentials.json', 
    SCOPES
)

# Run the flow - this will open a browser for authentication
print("🌐 Opening browser for authentication...")
creds = flow.run_local_server(port=0)

# Build the Calendar service
service = build('calendar', 'v3', credentials=creds)

print("✅ Successfully authenticated!")

In [ ]:
from datetime import datetime, time
from zoneinfo import ZoneInfo

def get_events_for_date(service, date):
    """
    date: datetime.date (e.g., date(2025, 1, 15))
    returns: list of events for that day in the calendar's timezone
    """

    # Get calendar timezone
    calendar = service.calendars().get(calendarId='primary').execute()
    tz = ZoneInfo(calendar['timeZone'])

    # Start and end of the given day in calendar timezone
    start_of_day = datetime.combine(date, time.min, tzinfo=tz).isoformat()
    end_of_day = datetime.combine(date, time.max, tzinfo=tz).isoformat()

    # Fetch events
    events_result = service.events().list(
        calendarId='primary',
        timeMin=start_of_day,
        timeMax=end_of_day,
        singleEvents=True,
        orderBy='startTime'
    ).execute()

    return events_result.get('items', [])


In [ ]:
from datetime import date

events = get_events_for_date(service, date(2025, 12, 27))
events = get_events_for_date(service, date(2025, 12, 26))

for event in events:
    start = event['start'].get('dateTime', event['start'].get('date'))
    end = event['end'].get('dateTime', event['end'].get('date'))
    status = event['status']
    # formatting the output for readability
    time_part = (start.split('T')[1][:5] +  " - " + end.split('T')[1][:5]) if 'T' in start else "All Day"
    print(f"{time_part} - {event.get('summary', '(No Title)')} - {status}")


In [ ]:
from datetime import datetime, timedelta

GRAY_COLOR_ID = "8"

for event in events:
    if 'dateTime' not in event['start']:
        continue  # skip all-day events

    title = event.get('summary', 'Untitled event')
    start = datetime.fromisoformat(event['start']['dateTime'])
    end = datetime.fromisoformat(event['end']['dateTime'])

    answer = input(f"Reserve transport time for '{title}'? (y/n): ").strip().lower()
    if answer != 'y':
        continue

    minutes = int(input("Minutes needed for transport: "))
    delta = timedelta(minutes=minutes)

    # Transport BEFORE
    before_event = {
        'summary': f"{title} - Transport",
        'start': {'dateTime': (start - delta).isoformat()},
        'end': {'dateTime': start.isoformat()},
        'colorId': GRAY_COLOR_ID
    }

    # Transport AFTER
    after_event = {
        'summary': f"{title} - Transport",
        'start': {'dateTime': end.isoformat()},
        'end': {'dateTime': (end + delta).isoformat()},
        'colorId': GRAY_COLOR_ID
    }

    service.events().insert(calendarId='primary', body=before_event).execute()
    service.events().insert(calendarId='primary', body=after_event).execute()

    print(f"Transport events created for '{title}'")
